In [1]:
import os
from decouple import config
from openai import OpenAI

OPENAI_API_KEY = config("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
UPSTASH_VECTOR_REST_URL = config("UPSTASH_VECTOR_REST_URL")
UPSTASH_VECTOR_REST_TOKEN = config("UPSTASH_VECTOR_REST_TOKEN")


In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import UpstashVectorStore

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
store = UpstashVectorStore(
    embedding=embeddings,
    index_url=UPSTASH_VECTOR_REST_URL,
    index_token=UPSTASH_VECTOR_REST_TOKEN
)


In [5]:
retriever = store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}
)

In [6]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI


In [7]:
LLM_CONFIG = {
    "model": "gpt-4o-mini",
    "api_key": OPENAI_API_KEY
}


In [8]:
llm = ChatOpenAI(**LLM_CONFIG)

In [9]:
from langchain_core.prompts import ChatPromptTemplate

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

In [10]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] * 123121,
)

runnable.invoke({"num": 31})

{'passed': {'num': 31}, 'modified': 3816751}

In [11]:

from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [12]:
chain={"context":retriever, "question":RunnablePassthrough()} | prompt | llm | parser

In [13]:
chain.invoke("what is the best food to eat in uttara")

'The best food to eat in Uttara can be found at the many street food vendors located near Sector 7 Bridge, which is a popular destination for street food lovers. Additionally, Uttara is home to various posh restaurants and fine dining establishments.'